# Integrate existing data sets on transmission experiments

In [1]:
from pathlib import Path
import pandas as pd
from mappings import Mappings
from ontology import Ontology
from dataset import Dataset

## Provide files ##

**Dataset**

In [2]:
data_dir = Path('../../../data/examples')

In [15]:
#trans_data = Path.cwd()/'data_maldi_UU_2_updated.xlsx'
trans_data_original = data_dir/'sample1_original.csv'
trans_data = data_dir/'sample1_modified.csv'
trans_data2 = data_dir/ 'sample2.csv'

**Mapping file**

In [24]:
config1 = data_dir/'sample1_mapping.yml'
config2 = data_dir/'sample2_mapping.yml'


**Ontology schema**

In [33]:
ont_file = Path.cwd()/'infection_trans.owl'

# Example 1


### Original dataset

In [6]:
dataset_original = pd.read_csv(trans_data_original)

dataset_original

,chicken_id,type,sex,pen,house,D3,D4 12:00,D4 16:00
0,12345,I,F,10,1,-,-,+


* The samples are taken on day 3 (no hour is specified), and 2 samples at day 4 (12:00 and 16:00). 

* We need to update the data in order to be able to create triples:

    1. The day of the sample in separate column -> create new column for each sample with day value

    2. If one of the samemples has specified an hour, others should specify as well -> we need to specify hour for sample taken on day3
    3. We should understand that sample result of one sample is related to the sample day and hour based on column names -> use sample's ordered number for column names. For example, sample1_date, sample2_date ...

## Corresponding preprocessed dataset

#### the step of renaming columns  and creating new ones should be manully preformed before creating the linked triples

In [7]:
dataset = pd.read_csv(trans_data)

dataset

,chicken_id,type,sex,pen,house,sample1_day,sample1_hour,sample1_result,sample2_day,sample2_hour,sample2_result,sample3_day,sample3_hour,sample3_result
0,12345,I,F,10,1,3,12:00,-,4,12:00,-,4,16:00,+


## Tidy dataset

### The script takes preprocessed data and mapping file to create tidy version (one date per row)

### Config example

In [8]:
mappings = Mappings(config1)

In [9]:
#dataset = pd.read_excel(trans_data).fillna(method='ffill',axis=0).fillna('')
dataset = Dataset(str(trans_data), mappings)

In [10]:
dataset.tidy_dataset

,chicken_id,house,pen,sex,type,experimentDay,experimentHour,sample.*_result
0,12345,1,10,F,I,3,12:00,-
1,12345,1,10,F,I,4,12:00,-
2,12345,1,10,F,I,4,16:00,+


In [11]:
columns = dataset.columns.values.tolist()

AttributeError: 'list' object has no attribute 'values'

## Ontology population

In [34]:
ontology = Ontology(ont_file)

**Create instances and populate ontology**
for each row of a dataset we create triples based on the mapping file

In [36]:
for _, row in dataset.tidy_dataset.iterrows():
    # check for the required field
    # if it doesn't exit then run it for each row
    if mappings.required_field is None or row[mappings.required_field]:
        ontology.populate_ontology(mappings, row)

### Quering the populated ontology

In [50]:
query1 = """
Prefix : <http://www.purl.org/infection_trans#>
SELECT ?ex_day ?ex_hour ?host_id ?inoculationStatus ?sample_type  ?sample_result
Where{
?experiment a :Experiment;
                          :experimentDay ?ex_day;
                          :hasMeasurement ?measurement.
              ?measurement a :Measurement;
                           :hasHost ?host.
                 ?measurement  :hasSample ?sample.
                optional {?measurement :experimentHour ?ex_hour.}
  
              ?host :id ?host_id;
                    :locatedIn ?env.
      optional{?host :treatment ?treatment.}
      optional{?host   :inoculationStatus ?inoculationStatus.}
      optional{?host   :type ?type.}
			optional{   ?sample :hasType ?sample_type.}
		  optional {?sample :result ?sample_result.}
      

}"""

qres1 = ontology.graph.query(query1)

## Query result

In [51]:


result = pd.DataFrame(
        data=([None if x is None else x.toPython() for x in row] for row in qres1),
        columns=[str(x) for x in qres1.vars],
    )

result

,ex_day,ex_hour,host_id,inoculationStatus,sample_type,sample_result
0,3,12:00,12345,None,http://www.purl.org/infection_trans#Swab,-
1,4,12:00,12345,None,http://www.purl.org/infection_trans#Swab,-
2,4,16:00,12345,None,http://www.purl.org/infection_trans#Swab,+


# Example 2

###  Dataset with multiple measurements per row 

In [25]:
mappings2 = Mappings(config2)

In [26]:
dataset2 = Dataset(str(trans_data2),mappings2)
dataset2.dataset

,house,pen,house_pen,treatment,animalnr_col,"I,S1, S2",weight_d0,weight_d21,BS0,BS0_date,...,swab1,swab1_date,swab2,swab2_date,swab2_value,swab3,swab3_date,swab3_value,value_weight_d21,value_weight_d0
0,1,S1,H1_S1,control,3_Ge,S2,"35,72",732,-*,-1,...,-,5,-*,6,,-*,7,,21,0
1,1,S1,H1_S1,control,10_Ge,S2,"43,51",934,-*,-1,...,-,5,-,6,,-,7,,21,0


In [27]:
dataset2.tidy_dataset

,"I,S1, S2",animalnr_col,house,house_pen,pen,treatment,experimentDay,weight_d.*,BS.*,BS.*_value,swab.*,swab.*_value
0,S2,3_Ge,1,H1_S1,S1,control,0,"35,72",,,,
1,S2,10_Ge,1,H1_S1,S1,control,0,"43,51",,,,
2,S2,3_Ge,1,H1_S1,S1,control,21,732,,,,
3,S2,10_Ge,1,H1_S1,S1,control,21,934,,,,
4,S2,3_Ge,1,H1_S1,S1,control,-1,,-*,,,
5,S2,10_Ge,1,H1_S1,S1,control,-1,,-*,,,
6,S2,3_Ge,1,H1_S1,S1,control,2,,-*,,,
7,S2,10_Ge,1,H1_S1,S1,control,2,,-*,,,
8,S2,3_Ge,1,H1_S1,S1,control,5,,,,-,
9,S2,10_Ge,1,H1_S1,S1,control,5,,,,-,
